<a href="https://colab.research.google.com/github/bryandaetz1/Travel_Recommendations/blob/master/hotel_recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [178]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [179]:
df = pd.read_csv('https://raw.githubusercontent.com/bryandaetz1/Travel_Recommendations/master/booking_com-travel_sample.csv')

In [180]:
df.head()

,address,city,country,crawl_date,hotel_brand,hotel_description,hotel_facilities,hotel_star_rating,image_count,latitude,locality,longitude,pageurl,property_id,property_name,property_type,province,qts,room_count,room_type,similar_hotel,site_review_count,site_review_rating,site_stay_review_rating,sitename,special_tag,state,uniq_id,zone
0,"KHIRSU, 246147 Pauri, India – Great location -",pauri,India,2016-09-01,NaN,Khirsu By GMVN offers accommodation in Pauri. ...,Bathroom:Toilet paper|Linen|Towels|Bathroom|To...,NaN,3.0,30.123749,NaN,78.793774,http://www.booking.com/hotel/in/khirsu-by-gmvn...,1742658.0,Khirsu By GMVN,204,NaN,2016-09-01 11:52:38 +0000,18.0,Economy Double Room,Hotel Mandakini,NaN,NaN,NaN,http://www.booking.com/,Share,Uttarakhand,a5ea72415f8007fcbe65759830fdd3d9,NaN
1,"Kaathadimattam, Balacola Post, NEAR Siva Tea F...",ooty,India,2016-09-01,NaN,"Situated in Ooty in the Tamil Nadu Region, 8 k...",Bathroom:Toilet paper|Linen|Towels|Bidet|Towel...,3 stars,NaN,11.329595,NaN,76.686732,http://www.booking.com/hotel/in/global-village...,1755986.0,Global Village Ooty,212,NaN,2016-09-01 11:52:38 +0000,10.0,British Empire Chalet,Treebo Yantra Leisures|Western Valley Resorts|...,5,7.6,Location:8.5|Staff:8|Cleanliness:7.5|Comfort:7...,http://www.booking.com/,Share,Tamil Nadu,7e0b055417271bbd9dae902f3e231ed4,NaN
2,"PIPALKOTI, 246472 Pīpalkoti, India – Show map",pīpalkoti,India,2016-09-01,NaN,TRH Pipalkoti offers accommodation in Pīpalkot...,Bathroom:Toilet paper|Linen|Towels|Bathroom•Vi...,NaN,4.0,30.429540,NaN,79.428116,http://www.booking.com/hotel/in/trh-pipalkoti....,1741747.0,TRH Pipalkoti,204,NaN,2016-09-01 11:52:38 +0000,28.0,Economy Double Room,TRH Joshimath (New),NaN,NaN,NaN,http://www.booking.com/,Share,Uttarakhand,72c0af09827bbb620365aa5df523ba1d,NaN
3,"1 KARIYIL HOUSE KUMARAKOM NORTH PO KOTTAYAM, 6...",kumarakom,India,2016-09-01,NaN,"Swasti house boat 2 is located in Kumarakom, 3...",Bathroom:Toilet paper|Towels|Bath|Shower•Bedro...,NaN,2.0,9.616057,NaN,76.428269,http://www.booking.com/hotel/in/swasti-house-b...,1747392.0,Swasti house boat 2,204,NaN,2016-09-01 11:52:38 +0000,1.0,Deluxe Room,NaN,NaN,NaN,NaN,http://www.booking.com/,Share,Kerala,eb6fd33d99aa4a8088caa8f3ecb08275,NaN
4,"Kavanattinkara, 686563 Kumarakom, India – Show...",kumarakom,India,2016-09-01,NaN,"Amrutham Houseboat 2 is set in Kumarakom, 5 km...",Bathroom:Toilet paper|Linen|Towels|Towels/Shee...,NaN,NaN,9.632854,NaN,76.423570,http://www.booking.com/hotel/in/amrutham-house...,1744962.0,Amrutham Houseboat 2,215,NaN,2016-09-01 11:52:38 +0000,2.0,Mobile Home,Mandala Beach House & Cottages,NaN,NaN,NaN,http://www.booking.com/,Share,Kerala,0814d9af7ad808863c04db8f30437c57,NaN


In [181]:
df['hotel_description'].head()

0    Khirsu By GMVN offers accommodation in Pauri. ...
1    Situated in Ooty in the Tamil Nadu Region, 8 k...
2    TRH Pipalkoti offers accommodation in Pīpalkot...
3    Swasti house boat 2 is located in Kumarakom, 3...
4    Amrutham Houseboat 2 is set in Kumarakom, 5 km...
Name: hotel_description, dtype: object

In [182]:
df['hotel_description'][0]

'Khirsu By GMVN offers accommodation in Pauri. Each room has a TV and offers views of the mountains. Some rooms have a seating area where you can relax. Rooms have a private bathroom. Khirsu By GMVN features free WiFi . You will find room service at the property. The hotel also offers car hire. The nearest airport is Jolly Grant Airport, 59 km from Khirsu By GMVN.'

In [183]:
df['country'].value_counts()

India    6000
Name: country, dtype: int64

In [184]:
df['city'].value_counts()

new delhi       309
bangalore       193
mumbai          172
jaipur          163
chennai         152
               ... 
bāghdogra         1
gangāpur          1
tiruchengodu      1
udupi             1
amrāvati          1
Name: city, Length: 543, dtype: int64

In [185]:
#checking for missing values
df.isna().sum()

address                     590
city                         18
country                       0
crawl_date                    0
hotel_brand                4688
hotel_description           590
hotel_facilities            590
hotel_star_rating          2925
image_count                2463
latitude                      0
locality                   5471
longitude                     0
pageurl                       0
property_id                 590
property_name               590
property_type                 0
province                   5471
qts                           0
room_count                  590
room_type                     5
similar_hotel              1755
site_review_count          1798
site_review_rating         1208
site_stay_review_rating    3837
sitename                      0
special_tag                 590
state                        49
uniq_id                       0
zone                       5471
dtype: int64

In [186]:
#dropping rows with missing hotel name since the recommendation function will be using the hotel name for recommendations
df.drop(df[df['property_name'].isna()].index, axis = 0, inplace = True)

In [187]:
#checking to see if this also removed rows where hotel description was missing
df[['property_name','hotel_description']].isna().sum()

property_name        0
hotel_description    0
dtype: int64

In [188]:
df.shape

(5410, 29)

In [189]:
df['hotel_description'].nunique()

2266

In [190]:
df['property_name'].nunique()

2054

**Dropping Duplicates:**   
Chose to drop duplicate rows based on hotel description rather than property name since there are more unique hotel descriptions than names.  It is possible that there are two different hotels with the same name, but it would be far less likely for two different hotels to have the exact same non-null description.

In [191]:
#dropping duplicates based on hotel description 
df = df.drop_duplicates(subset = 'hotel_description', keep = 'first').reset_index(drop = True)

In [192]:
df.shape

(2266, 29)

In [193]:
#define TF-IDF vectorizer object, remove all english stopwords (i.e. 'the', 'a', etc.)
tfidf = TfidfVectorizer(stop_words = 'english')

#contruct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df['hotel_description'])

tfidf_matrix.shape

(2266, 5249)

In [194]:
#creating cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

cosine_sim.shape

(2266, 2266)

In [195]:
indices = pd.Series(df.index, index = df['property_name']).drop_duplicates()
indices[:10]

property_name
Khirsu By GMVN                  0
Global Village Ooty             1
TRH Pipalkoti                   2
Swasti house boat 2             3
Amrutham Houseboat 2            4
The Hill Side                   5
Hotel Mountain Face             6
Rahi Motel Haridwar             7
Swasti house boat 4             8
The Gaff in Kalimpong-Deluxe    9
dtype: int64

In [202]:
# function that takes hotel name as input and outputs similar hotels based on description
def get_recommendations(hotel, cosine_sim=cosine_sim):
  #get the index of the hotel that matches the name
  idx = indices[hotel]

  #get the pairwise similarity scores of all hotels with that hotel
  sim_scores = list(enumerate(cosine_sim[idx]))

  #sort the hotels based on similarity scores
  sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)

  #get the scores of the 10 most similar hotels (excluding the one you enter as an input)
  sim_scores = sim_scores[1:11]
  
  #get the hotel indices
  hotel_indices = [i[0] for i in sim_scores]

  #return top 10 most similar hotels
  return df['property_name'].iloc[hotel_indices]


In [205]:
get_recommendations('Khirsu By GMVN')

1764                   Hotel Grand Plaza
12                           Grand Hotel
1208                   Hotel N.M.R.Plaza
39                 Hotel Royal Residency
142                         Ekant Resort
185                    Home In The Hills
1533    Vista Rooms near Railway Station
7                    Rahi Motel Haridwar
1418                         Hotel Surya
88                          Hotel Chawla
Name: property_name, dtype: object

In [206]:
df[['property_name','similar_hotel']][df['property_name'] == 'Khirsu By GMVN']

,property_name,similar_hotel
0,Khirsu By GMVN,Hotel Mandakini
